# Supercloud Customer

A Microsoft Azure Supercloud customer is defined as a customer who has purchased at least one product from every product category listed in the `products` table.

Write a query that identifies the customer IDs of these Supercloud customers.

# Answer

There are two tables I will be working with, `customer_contracts` & `products`. This is the `customer_contracts` table:

```
CREATE TABLE customer_contracts (
	customer_id smallint,
	product_id smallint,
	amount smallint
);

COPY customer_contracts
FROM '/YourDirectory/customer_contracts.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM customer_contracts;
```

<img src = "customer_contracts Table.png" width = "600" style = "margin:auto"/>

This is the `products` table:

```
CREATE TABLE products (
	product_id smallserial,
	product_category varchar(10),
	product_name text
);

INSERT INTO products (product_category, product_name)
VALUES ('Analytics', 'Azure Databricks'),
	   ('Analytics', 'Azure Stream Analytics'),
	   ('Containers', 'Azure Kubernetes Service'),
	   ('Containers', 'Azure Service Fabric'),
	   ('Compute', 'Virtual Machines'),
	   ('Compute', 'Azure Functions');

SELECT * FROM products;
```

<img src = "products Table.png" width = "600" style = "margin:auto"/>

I will join the `products` table onto the `customer_contracts` table. This seems like a problem for `crosstab()`.

```
CREATE EXTENSION tablefunc;

SELECT * 
FROM crosstab(
	'SELECT cc.customer_id, 
	 	    products.product_category,
		    count(*)
	 FROM customer_contracts AS cc
	 LEFT JOIN products
	 	 ON cc.product_id = products.product_id
	 GROUP BY customer_id, product_category
	 ORDER BY customer_id',
	
	'SELECT product_category
	 FROM products
	 GROUP BY product_category
	 ORDER BY product_category'
) AS (
	customer_id smallint,
	analytics smallint,
	compute smallint,
	containers smallint
)
WHERE ROW(analytics, compute, containers) IS NOT NULL;
```

The `crosstab()` creates a pivot table that counts the number of products each customer bought within each `product_category`, & the filter finds the customer who bought at least 1 item in each of the product categories.

<img src = "Supercloud Customer.png" width = "600" style = "margin:auto"/>

Customer 7 is a supercloud customer.